# MÉTRICA 3_4_07 — Vehículos en préstamo o comodato
| Escenario                          | Condición                                        | Resultado     |
| ---------------------------------- | ------------------------------------------------ | ------------- |
| No hay préstamos/comodatos         | No existe el arreglo `prestamoComodato.prestamo` | 0 (CUMPLE)    |
| No hay vehículos                   | Ningún préstamo tiene `tipoBien = VEHICULO`      | 0 (CUMPLE)    |
| ≤ 2 vehículos en préstamo/comodato | Conteo de vehículos ≤ 2                          | 0 (CUMPLE)    |
| > 2 vehículos en préstamo/comodato | Conteo de vehículos > 2                          | 1 (NO_CUMPLE) |



In [ ]:
import traceback
import unicodedata
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

METRIC_ID = "3_4_07_VEHICULOS_PRESTAMO_COMODATO"
BATCH_SIZE = 10000

# Tipos de bienes considerados vehículo
TIPOS_VEHICULO = {
    "VEHICULO", "VEHÍCULO", "AUTO", "AUTOMOVIL", "CAMIONETA", "MOTO", "MOTOCICLETA"
}

def normalizar(texto):
    if not texto:
        return ""
    texto = texto.upper()
    texto = unicodedata.normalize("NFD", texto)
    return "".join(c for c in texto if unicodedata.category(c) != "Mn")


def extraer_lista(doc, path):
    try:
        partes = path.split(".")
        actuales = [doc]
        for p in partes:
            siguientes = []
            for a in actuales:
                if isinstance(a, dict):
                    v = a.get(p)
                    if v is None:
                        continue
                    if isinstance(v, list):
                        siguientes.extend(v)
                    else:
                        siguientes.append(v)
                elif isinstance(a, list):
                    for sub in a:
                        if isinstance(sub, dict):
                            v = sub.get(p)
                            if v is None:
                                continue
                            if isinstance(v, list):
                                siguientes.extend(v)
                            else:
                                siguientes.append(v)
            actuales = siguientes
        return actuales
    except:
        return []


def evaluar_metrica(doc):
    prestamos = extraer_lista(doc, "declaracion.situacionPatrimonial.prestamoComodato.prestamo")

    if not prestamos:
        # No préstamos → no anomalía
        return 0

    conteo_vehiculos = 0

    for p in prestamos:
        tipo = normalizar(p.get("tipoBien", ""))
        if tipo in TIPOS_VEHICULO:
            conteo_vehiculos += 1

    # Regla: vehículos en comodato <= 2 → CUMPLE (0)
    return 1 if conteo_vehiculos > 2 else 0


def procesar_metrica():
    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]
    src = db[SOURCE_COLLECTION_NAME]
    tgt = db[METRICS_COLLECTION_NAME]

    total_docs = src.estimated_document_count()
    print(f"🚗 Procesando {METRIC_ID} en {total_docs:,} documentos...")

    skip = 0
    processed = 0
    lote = 0

    try:
        while True:
            cursor = list(
                src.find(
                    {},
                    {
                        "_id": 1,
                        "declaracion.situacionPatrimonial.prestamoComodato.prestamo": 1
                    }
                ).skip(skip).limit(BATCH_SIZE)
            )

            if not cursor:
                break

            ops = []

            for doc in cursor:
                try:
                    res = evaluar_metrica(doc)
                except:
                    res = 0

                ops.append(UpdateOne(
                    {"_id": doc["_id"]},
                    {"$set": {METRIC_ID: res}}
                ))

            tgt.bulk_write(ops)

            processed += len(ops)
            lote += 1

            print(f" 🔄 Lote {lote:,}: {processed:,}/{total_docs:,} procesados", flush=True)

            skip += BATCH_SIZE

        print("\n✅ Proceso completado correctamente.")

    except Exception:
        traceback.print_exc()

    finally:
        client.close()
        print("🔒 Conexión cerrada.")


if __name__ == "__main__":
    procesar_metrica()


🚗 Procesando 3_4_07_VEHICULOS_PRESTAMO_COMODATO en 8,942,592 documentos...
 🔄 Lote 1: 10,000/8,942,592 procesados
 🔄 Lote 2: 20,000/8,942,592 procesados
 🔄 Lote 3: 30,000/8,942,592 procesados
 🔄 Lote 4: 40,000/8,942,592 procesados
